In [1]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
from matplotlib import pyplot as plt
import sklearn.metrics as sm

In [2]:
C_Lambda = 0.03
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
M = 10
PHI = []
IsSynthetic = False

In [3]:
# Extract target (relavance values) into a vector
def GetTargetVector(filePath):
    t = []
    with open(filePath, 'rU') as f:
        reader = csv.reader(f)
        for row in reader:  
            t.append(int(row[0]))
    #print("Raw Training Generated..")
    return t

# Extract data(input values) into a vector
def GenerateRawData(filePath, IsSynthetic):    
    dataMatrix = [] 
    with open(filePath, 'rU') as fi:
        reader = csv.reader(fi)
        for row in reader:
            dataRow = []
            for column in row:
                dataRow.append(float(column))
            dataMatrix.append(dataRow)   
    
    if IsSynthetic == False :
        dataMatrix = np.delete(dataMatrix, [5,6,7,8,9], axis=1)
    dataMatrix = np.transpose(dataMatrix)     
    #print ("Data Matrix Generated..")
    return dataMatrix

# Partition the target(relavance values) vector as 80% of total for training
def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    return t

# Partition the data(input values) vector as 80% of total for training
def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    return d2

# Partition the data(input values) vector as 10% of total for validation set
def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    return dataMatrix

# Partition the target(relavance values) vector as 10% of total for validation set
def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    return t

# Calculating the variance of each feature amoong the data(input values) and storing it in a matrix. This is required for 
# calculation of basis functions
def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    return BigSigma

# Intermediate calculations of Gaussian Radial basis function
def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

# Calculation of Gaussian Radial basis function using it's vector form of formula
def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

# Calculation of Basis function matrix from Gaussian Radial basis function values
def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    return PHI

# Calculation of weight vector using the derived form of original linear regression equation.
def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    return W

# Calculation of target (relevance value) for test set using the linear regression formula
def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    return Y

# Calculation of Root mean square error and accuracy for any two vectors 
# Given predicted values vector and actual target vector, the function would return accuracy and Root mean square error
def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

## Fetch and Prepare Dataset

In [4]:
# Fetching data(input values) and target(relevance values) from csv to matrices
RawTarget = GetTargetVector('Querylevelnorm_t.csv')
RawData   = GenerateRawData('Querylevelnorm_X.csv',IsSynthetic)

C:\Users\MoMo\Anaconda3\envs\ML Env\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.
C:\Users\MoMo\Anaconda3\envs\ML Env\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: 'U' mode is deprecated
  


## Prepare Data

In [5]:
# Preparing Training data set
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent))
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent)

# Preparing Validation data set
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget))))
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))

# Preparing Test data set
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))

## Closed Form Solution [Finding Weights using Moore- Penrose pseudo- Inverse Matrix]

In [6]:
ErmsArr = []
AccuracyArr = []

# K-means is used to find the number of clusters we can generate from the given data(input values) 
# Because we want to introduce non-linearity  
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingData))
Mu = kmeans.cluster_centers_

# Calculations for the formula: y(x, w) = wTφ(x)
#  where φ(x) = Gaussian radial basis function
#        wT = Weight vector
#        y(x, w) = Target(Relevance values)
#  We calculate W using TRAINING_PHI and Training target(relevance values) and 
#  later use this W for calculating Test and Validation target(relevance values)
BigSigma     = GenerateBigSigma(RawData, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(RawData, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestData, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValData, Mu, BigSigma, 100)

In [7]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(10, 41)
(41, 41)
(55699, 10)
(10,)
(6962, 10)
(6961, 10)


## Finding Erms on training, validation and test set 

In [8]:
# Getting output(relevance value) for Training, validation and test set
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

# Accuracy and Root mean square error calculation
TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))

# R-score is regression score function 
Training_rScore = sm.r2_score(TrainingTarget, TR_TEST_OUT)
Validation_rScore = sm.r2_score(ValDataAct,VAL_TEST_OUT)
Testing_rScore  = sm.r2_score(TestDataAct, TEST_OUT)

In [9]:
print ('UBITname      = mkheraja')
print ('Person Number = 50290424')
print ("-------Closed Form with Radial Basis Function-------")
print ("M = ", M, "\nLambda =", C_Lambda)

# R-score is regression score function
print("R-score for Training:", round(Training_rScore, 5))
print("R-score for Validation:", round(Validation_rScore, 5))
print("R-score for test:", round(Testing_rScore, 5))

# Root mean square error 
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))

# Accuracy
print ("Accuracy Training   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("Accuracy Validation = " + str(float(ValidationAccuracy.split(',')[0])))
print ("Accuracy Testing    = " + str(float(TestAccuracy.split(',')[0])))

UBITname      = mkheraja
Person Number = 50290424
-------Closed Form with Radial Basis Function-------
M =  10 
Lambda = 0.03
R-score for Training: 0.05217
R-score for Validation: 0.05408
R-score for test: 0.02252
E_rms Training   = 0.549469406713785
E_rms Validation = 0.5384281741393284
E_rms Testing    = 0.6279788453828589
Accuracy Training   = 73.92233253738846
Accuracy Validation = 74.6768170066073
Accuracy Testing    = 69.87501795719005


## Gradient Descent solution for Linear Regression

In [ ]:
# we initialize weight vector randomly
W_Now        = np.dot(220,W)
# Regularization term for weight decay factor 
La           = 2
learningRate = 0.01
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []

L_Accuracy_TR = 0.0
L_Accuracy_Val = 0.0
L_Accuracy_Test = 0.0

# Iterating over data(input values)
for i in range(0,400):
    
    # Updating weight vector iteratively over data(input values) by adding Delta_W each time
    # where Delta_W is the negative product of learning rate and Delta_E
    # where Delta_E is sum of Delta_E_D and La_Delta_E_W
    # where Delta_E_D is 
    # where La_Delta_E_W is 
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    L_Accuracy_TR = L_Accuracy_TR + float(Erms_TR.split(',')[0])

    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    L_Accuracy_Val = L_Accuracy_Val + float(Erms_Val.split(',')[0])

    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))
    L_Accuracy_Test = L_Accuracy_TR + float(Erms_Test.split(',')[0])

In [ ]:
print ('----------Gradient Descent Solution--------------------')
print ("M =", M, " \nLambda  =", La,"\nlearningRate(eta)", learningRate)

Training_accuracy  = sm.r2_score(TrainingTarget, TR_TEST_OUT)
print("R-score for Training:", round(Training_accuracy, 2))
print ("E_rms Training   = " + str(np.around(min(L_Erms_TR),5)))
# print("Training accuracy = ", L_Accuracy_TR)

Training_accuracy  = sm.r2_score(ValDataAct, VAL_TEST_OUT)
print("R-score for Validation:", round(Training_accuracy, 2))
print ("E_rms Validation = " + str(np.around(min(L_Erms_Val),5)))
# print("Validation accuracy = ", L_Accuracy_Val)

Training_accuracy  = sm.r2_score(TestDataAct, TEST_OUT)
print("R-score for test:", round(Training_accuracy, 2))
print ("E_rms Testing    = " + str(np.around(min(L_Erms_Test),5)))
# print("Testing accuracy = ", L_Accuracy_Test)